# Image API

This notebook provides an overview of PHL-Microsat's Image API endpoint: A catalog of Diwata-1 images.

The ImageAPI has a couple of endpoints available:
* [/scenes](#scenes)
* [/scenes/histogram](#histogram)
* [/scenes/geohash](#geohash)


## GET /scenes <a class="anchor" id="scenes"></a>

This endpoint serves a list of satellite images.

### Fetch Data 
Let's make a request using the default parameters

In [ ]:
import requests
r = requests.get('https://api.orange.phl-microsat.xyz/scenes')
r.json()

### Inspect Results

The response returns a JSON file with two parts, the data and the meta. Let's look at the data part.

In [ ]:
data = r.json()['data']
data

### Row Contents

The data returns a list of dictionaries, each with information about a specific image. Each dictionary has the following information

* scene_id
* scene_center
* scene_footprint
* satellite_name
* payload
* cloudcover
* capture_time
* wavelength
* published_time
* links
* image_quality
* bands
* day_or_night
* products
* satellite_color
* payload_color
* mission_id
* mission_name
* mission_start_time 

### Inspect a Sample Image

In [ ]:
sample = data[0]
sample

#### Setup Folium

To visualize shapes, we'll be using **Folium**, a Python library for plotting maps using `Leaflet.js`. Here's an example of a map setup using Folium.

In [ ]:
import folium

COUNTRY_CENTER = [12.8797, 121.7740]
m = folium.Map(location=COUNTRY_CENTER, zoom_start=6)
m

### Geometries

There are two geometry attributes included in each item: `scene_footprint` and `scene_center`

#### Geometry: Scene Footprint

This is the actual area covered by the image on the ground. It is represented as a GeoJSON polygon geometry.

In [ ]:
footprint = sample['scene_footprint']
footprint

Let's plot the scene footprint:

In [ ]:
folium.GeoJson(
    footprint,
    name='footprint'
).add_to(m)
m

#### Geometry: Scene Center

It's the centroid of the footprint on the ground. It is represented as a GeoJSON point.

In [ ]:
scene_center = sample['scene_center']
scene_center

Let's plot the scene center:

In [ ]:
m = folium.Map(location=COUNTRY_CENTER, zoom_start=5)

folium.GeoJson(
    scene_center,
    name='scene_center'
).add_to(m)

m

### Filters

The API also allows filtering of results. Feel play with some filters and map the results using Folium.

#### Basic Filters
| **Parameter**            | **Description**                                                                                           | **Example**                                           |
|--------------------------|-----------------------------------------------------------------------------------------------------------|-------------------------------------------------------|
| payload                  | Filter by payload. Sample payloads are: 'hpt', 'smi', 'mfc'. Multiple allowed.                            | `/scenes?payload=smi,hpt`                             |
| satellite_name           | Filter by satellite name. Multiple allowed.                                                               | `/scenes?satellite_name=diwata-1,landsat8`             |
| since                    | Return scenes captured on or after specified date (UTC)                                                   | `/scenes?since=2016-01-01T00:00:00Z`                  |      
| until                    | Return scenes captured on or before specified date (UTC)                                                  | `/scenes?until=2016-01-01T00:00:00Z`                  |

#### Geographical Filters
| **Parameter**            | **Description**                                                                                           | **Example**                                           |
|--------------------------|-----------------------------------------------------------------------------------------------------------|-------------------------------------------------------|
| geo_bounding_box         | Returns scenes that intersect with a bounding box                                                         | `/scenes?geo_bounding_box=118.30078,16.10915,128.6718,9.34067` |
| geo_point                | Returns scenes that encloses a specified pinned point                                                     | `/scenes?geo_point=121.11328,15.05090`  |

---

## GET /scenes/histogram <a class="anchor" id="histogram"></a>

This endpoint returns the number of scenes per day/number of days. `bucket_size` indicates the number of days to group the results by.

In [ ]:
import requests

r = requests.get('https://api.orange.phl-microsat.xyz/scenes/histogram?bucket_size=7')
r.json()

Filters for the GET /scenes endpoint also work here!

---

## GET /scenes/geohash <a class="anchor" id="geohash"></a>

This endpoint returns the number of scenes grouped by geohash.

In [ ]:
r = requests.get('https://api.orange.phl-microsat.xyz/scenes/geohash')
r.json()